# Agents that can code

In [ ]:
# llm_config object to configure our agents
llm_config = {
    "model": "gpt-4o", 
    "api_key": ""
    }

## Command Line Executor

In this project, we will be generating simple code and will use setup that executes code locally (We can use the another version of the tool that relies on Docker that will create a virtualized version of a python machine to execute the code)

Import the codeExecutor class from autogen and setup a local executor environment for the agents.
We are going to specify that if the code hangs and nothing happens during 200 seconds we will interrupt it and that we want all the code to execute and provide results in the `coding` folder:

In [ ]:
from autogen.code_utils import create_virtual_env
from autogen.coding import CodeBlock, LocalCommandLineCodeExecutor

venv_dir = ".env_llm"
venv_context = create_virtual_env(venv_dir)

executor = LocalCommandLineCodeExecutor(
    virtual_env_context=venv_context,
    timeout=200,
    work_dir="coding",
)
print(
    executor.execute_code_blocks(code_blocks=[CodeBlock(language="python", code="import sys; print(sys.executable)")])
)

## Code executor agent

We will create two agents:
1. A **code_writer_agent**: this agent rely on chatGPT to generate python code to accomplish the requested task.
This agent is called `AssistantAgent` one of the subcategories of `ConverseableAgent`. The `AssistantAgent` is designed to write code.

In [ ]:
from autogen import AssistantAgent

# Agent that writes code
code_writer_agent = AssistantAgent(
    name="code_writer_agent",
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

This agent does not execute code (`code_execution_config=False`) and we do not need to specify a system prompt message for this agent because it already has a default one that is designed to suggest python code when it can help. We can see the default system prompt message using below:

In [ ]:
code_writer_agent_system_message = code_writer_agent.system_message
print(code_writer_agent_system_message)

2. Second type of agent is a **code_executor_agent**: this agent does not use an LLM (`llm_config=False`), this agent will simply execute code it was provided, locally in the `coding` folder, and send back the result of that code as a reply.  
This agent needs a `code_execution_config` to be able to execute code, and we will provide it with the one we defined earlier called `executor` (`code_execution_config={"executor": executor}`). We will also give it a default auto reply that it will use if the user does not have any additional details to provide. Basically, each time this agent will execute some code, it will request feedback from the user, and if the user does not provide any feedback, it will use the auto reply.

In [ ]:
from autogen import ConversableAgent

# Code executor agent
code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={"executor": executor},
    human_input_mode="ALWAYS",
    default_auto_reply=
    "Please continue. If everything is done, reply 'TERMINATE'.",
)

## Coding task

We will now ask the agents to perform a task that requires coding. The task will be provided to the Code Writer agent by the Code Executor agent, the Code Writer agent will then propose a code that should fulfill the task, the Code Executor agent will then execute that code and report back the resutl to the Code Writer. There might be several exchanges between these two agents until they accomplish the task.  
For example, if a code error is encountered, the error will be reported to the Code Writer who'll propose a corrected version of the code.

Let's prepare a simple task to test the code execution scheme. We want the agents to generate a plot that shows the price evolution of two assets, NVDA and MSFT for the last 4 years in a way that makes it easy for us to compare them. We will also add the 50 weeks moving average. Let's write it:

In [ ]:
import datetime

today = datetime.datetime.now().date()

message = f"Today is {today}. "\
"Create a plot showing the normalized price of NVDA and MSFT for the last 4 years "\
"with their 50 weeks moving average. "\
"Make sure the code is in a markdown code block, save the figure"\
" to a file asset_analysis.png and show it. Provide all the code necessary in a single python bloc. "\
"Re-provide the code block that needs to be executed with each of your messages. "\
"If python packages are necessary to execute the code, provide a markdown "\
"sh block with only the command necessary to install them and no comments."

During the execution, the executor might encounter some errors. For example, if you did not install the modules required by the code writer, you will get an error. The agents might also just install the missing modules themselves.

In [ ]:
chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message=message
)

The code will now execute.

Before executing the code provided by the Code Writer, user will be requested to provide feedback. That is an important safety feature. Always review the code written by an agent before executing it, there might be some malicious content. We can also provide feedback if something is wrong with the code. If not, we can just press Enter to let the Code Executor go ahead and execute the code.